In [1]:
from config import *
import pandas as pd
import pickle
from sklearn import metrics
from sklearn import linear_model, ensemble, tree, naive_bayes, neighbors, svm

In [2]:
train_df = pd.read_csv(train_processed)
test_df = pd.read_csv(test_processed)

y_train = train_df['Longevity']
x_train = train_df.drop(columns='Longevity')

y_test = test_df['Longevity']
x_test = test_df.drop(columns='Longevity')

In [3]:
x_train.head()

,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,3,0,73,1,0,725,1,1,0,6,3
1,1,1,75,1,0,7128,0,1,0,3,3
2,3,1,73,0,0,793,1,0,1,6,4
3,1,1,74,1,0,5310,0,1,0,2,2
4,3,0,74,0,0,805,1,0,1,6,4


# Utilities

In [4]:
def accuracy(y_test, y_pred):
    return round(metrics.accuracy_score(y_test, y_pred) * 100, 2)

# Логистическая регрессия

In [5]:
logreg = linear_model.LogisticRegression(solver='liblinear')
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
acc_log = accuracy(y_test, y_pred)

coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df['Correlation'] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)
coeff_df

,Feature,Correlation
0,Education,-0.535318
1,Sex,2.491259
2,Age,0.024130
3,Pet,-0.292914
4,Children,0.039262
5,Activity,0.000023
6,Sport,0.040278
7,Family,-0.253652
8,IsAlone,-0.475323
9,Age*Education,-0.146547


# Машина опорных векторов (SVC и LinearSVC)

In [6]:
svc = svm.SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)
acc_svc = accuracy(y_test, y_pred)

In [7]:
linear_svc = svm.LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_test)
acc_linear_svc = accuracy(y_test, y_pred)

/home/team06/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# k-Ближайшие соседи (KNN)

In [8]:
knn = neighbors.KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
acc_knn = accuracy(y_test, y_pred)

# Наивный байесовский классификатор

In [9]:
gausNb = naive_bayes.GaussianNB()
gausNb.fit(x_train, y_train)
y_pred = gausNb.predict(x_test)
acc_gaussian = accuracy(y_test, y_pred)

# Персептон

In [10]:
percep = linear_model.Perceptron()
percep.fit(x_train, y_train)
y_pred = percep.predict(x_test)
acc_perceptron = accuracy(y_test, y_pred)

# Стохастический градиентный спуск

Стохастический градиентный спуск (SGD) — это метод итерации для оптимизации целевой функции с подходящими свойствами гладкости. Его можно расценивать как стохастическую аппроксимацию оптимизации методом градиентного спуска, поскольку он заменяет реальный градиент (вычисленный из полного набора данных) путём оценки такового (вычисленного из случайно выбранного подмножества данных). Особенно в приложениях обработки больших данных это сокращает вычислительные ресурсы, достигая более быстрые итерации в обмен на более низкую скорость сходимости.

In [11]:
sgd = linear_model.SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)
acc_sgd = accuracy(y_test, y_pred)

# Дерево решений

Деревья решений - это это способ представления правил в иерархической виде, где каждому объекту соответствует единственный узел, пораждающий решение. Структура дерева представляет собой «листья» и «ветки». На рёбрах дерева решения записаны атрибуты, от которых зависит целевая функция, в «листьях» записаны значения целевой функции, а в остальных узлах - атрибуты, по которым различаются случаи.

Достоинством таких алгоритмов является высокая наглядность представления и простота интерпретации результатов, что может быть очень важным для предметной области: оказывается возможным не только провести процесс классификации, но и объяснить почему тот или иной объект отнесён к какому-либо классу.

In [12]:
dectr = tree.DecisionTreeClassifier()
dectr.fit(x_train, y_train)
y_pred = dectr.predict(x_test)
acc_decision_tree = accuracy(y_test, y_pred)

# Случайный лес

Случайный лес - модель, состоящая из множества деревьев решений. Вместо того,чтобы просто усреднять прогнозы разных деревьев (такая концепция называется просто «лес»), эта модель использует две ключевые концепции, которые и делают этот лес случайным.

Случайная выборка образцов из набора данных при построении деревьев.
При разделении узлов выбираются случайные наборы параметров.

In [13]:
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 50, max_depth = 10, random_state = 1)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)
acc_random_forest = accuracy(y_test, y_pred)

# Выбор лучшей модели

In [14]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
1,KNN,71.46
3,Random Forest,69.03
4,Naive Bayes,68.58
8,Decision Tree,67.70
2,Logistic Regression,67.48
0,Support Vector Machines,65.93
5,Perceptron,63.94
7,Linear SVC,63.94
6,Stochastic Gradient Decent,60.62


# Save binary models

In [15]:
pickle.dump(logreg, open(MODEL_DIR + 'log.pkl','wb'))
pickle.dump(rf_classifier, open(MODEL_DIR + 'random_forest.pkl','wb'))
pickle.dump(dectr, open(MODEL_DIR+ 'decision_tree.pkl', 'wb'))
pickle.dump(sgd, open(MODEL_DIR+ 'sgd.pkl', 'wb'))
pickle.dump(percep, open(MODEL_DIR+ 'perceptron.pkl', 'wb'))
pickle.dump(gausNb, open(MODEL_DIR+ 'gaussian.pkl', 'wb'))
pickle.dump(knn, open(MODEL_DIR+ 'knn.pkl', 'wb'))
pickle.dump(svc, open(MODEL_DIR+ 'svc.pkl', 'wb'))
pickle.dump(linear_svc, open(MODEL_DIR+ 'linear_svc.pkl', 'wb'))